# <center>Laboratorium Analiza i bazy danych </center>

## <center>Łączenie tabel, podzapytania i funkcje agregujące</center>

## Przykładowe tabele obrazujące łączenie

Do zobrazowania operacji łączenia zostaną użyte tabele:

```sql
CREATE TABLE shape_a (
    id INT PRIMARY KEY,
    shape VARCHAR (100) NOT NULL
);
 
CREATE TABLE shape_b (
    id INT PRIMARY KEY,
    shape VARCHAR (100) NOT NULL
);
```
 
Polecenie CREATE TABLE tworzy tabelę o zadanej nazwie i strukturze. Ogólna postać to:
```sql
CREATE TABLE tab_name (
    col_name1 data_type constrain,
    col_name1 data_type constrain,
    ...
);
```
Należy uzupełnić ją danymi:
```sql
INSERT INTO shape_a (id, shape)
VALUES
    (1, 'Trójkąt'),
    (2, 'Kwadrat'),
    (3, 'Deltoid'),
    (4, 'Traper');
 
INSERT INTO shape_b (id, shape)
VALUES
    (1, 'Kwadrat'),
    (2, 'Trójkąt'),
    (3, 'Romb'),
    (4, 'Równoległobok');
```
Komenda INSERT INTO pozwala na dodanie do tabeli rekordów. Ogólna postać to:

```sql
INSERT INTO tab_name (col1_name, col2_name2, ...) 
VALUES
    (val1_col1, val2_col2),
    (val2_col1, val2_col2),
    ...
```

## Inner join 

Jest to podstawowy rodzaj złączenie. Ten sposób złączenia wybiera  te wiersze, dla których warunek złączenia jest spełniony. W żadnej z łączonych tabel kolumna użyta do łączenia nie może mieć wartości NULL. 

#### Przykład:
```sql
SELECT
    a.id id_a,
    a.shape shape_a,
    b.id id_b,
    b.shape shape_b
FROM
    shape_a a
INNER JOIN shape_b b ON a.shape = b.shape;
```
W zapytaniu powyżej użyto *aliasów* nazw tabel i column wynikowych, jest to szczególnie przydatne przy długich nazwach tabel i wprowadza czytelność w zapytaniu.

#### Wynik:
|id_a|shape_a|id_b|shape_b|
|-|-|-|-|
|1|Trójkąt|2|Trójkąt|
|2|Kwadrat|1|Kwadrat|

## OUTER JOIN

Istnieją trzy rodzaje złączeń OUTER:
- LEFT OUTER JOIN,
- RIGHT OUTER JOIN,
- FULL OUTER JOIN.

### LEFT OUTER JOIN

Ten rodzaj złączenie zwróci wszystkie rekordy z lewej tablicy i dopasuje do nich rekordy z prawej tablicy które spełniją zadany warunek złączenia. Jeżeli w prawej tablicy nie występują rekordy spełnijące warunek złączenia z lewą w ich miejscu pojawią się wartości NULL.

#### Przykład 1:
```sql
SELECT
    a.id id_a,
    a.shape shape_a,
    b.id id_b,
    b.shape shape_b
FROM
    shape_a a
LEFT JOIN shape_b b ON a.shape = b.shape;
```
#### Wynik:
|id_a|shape_a|id_b|shape_b|
|-|-|-|-|
|1|Trójkąt|2|Trójkąt|
|2|Kwadrat|1|Kwadrat|
|3|Deltoid|NULL|NULL|
|4|Traper|NULL|NULL|

#### Przykład 2:
```sql
SELECT
    b.id id_b,
    b.shape shape_b,
    a.id id_a,
    a.shape shape_a   
FROM
    shape_b b
LEFT JOIN shape_a a ON a.shape = b.shape;
```
#### Wynik:
|id_a|shape_a|id_b|shape_b|
|-|-|-|-|
|1|Kwadrat|2|Kwadrat|
|2|Trójkąt|1|Trójkąt|
|3|Romb|NULL|NULL|
|4|Równoległobok|NULL|NULL|

### RIGHT OUTER JOIN

Działa jak left outer join z tym, że prawa tablica w zapytaniu jest brana w całości.

#### Przykład:
```sql
SELECT
    a.id id_a,
    a.shape shape_a,
    b.id id_b,
    b.shape shape_b
FROM
    shape_a a
RIGHT JOIN shape_b b ON a.shape = b.shape;
```

#### Wynik:
|id_a|shape_a|id_b|shape_b|
|-|-|-|-|
|2|Kwadrat|1|Kwadrat|
|1|Trójkąt|2|Trójkąt|
|NULL|NULL|3|Romb|
|NULL|NULL|4|Równoległobok|


### FULL OUTER JOIN

Jest złączeniem które zwraca:
- wiersze dla których warunek złączenia jest spełniony,
- wiersze z lewej tabeli dla których nie ma odpowiedników w prawej,
- wiersze z prawej tabeli dla których nie ma odpowiedników w lewej. 

#### Przykład:
```sql
SELECT
    a.id id_a,
    a.shape shape_a,
    b.id id_b,
    b.shape shape_b
FROM
    shape_a a
FULL JOIN shape_b b ON a.shape = b.shape;
```
|id_a|shape_a|id_b|shape_b|
|-|-|-|-|
|1|Trójkąt|2|Trójkąt|
|2|Kwadrat|1|Kwadrat|
|3|Deltoid"|NULL|NULL|
|4|Traper|NULL|NULL|
|NULL|NULL|3|Romb|
|NULL|NULL|4|Równoległobok|

## Podzapytania

Podzapytanie zagnieżdżone SELECT znajduje się wewnątrz zewnętrznego zapytania SELECT, np. po klauzuli WHERE, HAVING lub FROM. W przypadku tego rodzaju zapytań w pierwszej kolejności wykonywane są wewnętrzne zapytania SELECT, a ich wynik jest wykorzystywany do zewnętrznego zapytania SELECT. Stąd łatwo zuważyć, że mogą one służyć do poprawy wydajności obsługi zapytania. Należy dobierać podzapytania tak by najbardziej zagnieżdżone podzapytanie zawierało najmniejszy zbiór poszukiwań. 

#### Przykład:
Jeżeli chcemy znaleźć w bazie informację o tytułach filmów zwróconych w zadanym okresie możemy wykonać następujące zapytanie:
```sql
SELECT
   film_id,
   title
FROM
   film
WHERE
   film_id IN (
      SELECT
         inventory.film_id
      FROM
         rental
      INNER JOIN inventory ON inventory.inventory_id = rental.inventory_id
      WHERE
         return_date BETWEEN '2005-05-29'
      AND '2005-05-30'
   );
```

#### Wynik
|film_id|title|
|-|-|
|307|Fellowship Autumn|
|255|Driving Polish|
|388|Gunfight Moon|
|130|Celebrity Horn|
|563|Massacre Usual|
|397|Hanky October|
|...|...|

### Używanie podzapytań

Pod zapytania mogą być używane w :
- SELECT,
- UPDATE,
- DELETE,
- Funkcjach agregujących,
- Do definiowania tabel tymczasowych.

Używając podzapytań zapytania SQL szybko mogą stać się mało czytelne. Przez co będą trudne w zrozumieniu i późniejszym utrzymaniu. W celu analizy zapytań można użyć klauzuli __EXPLAIN__, która przeanalizuje zapytanie. Klauzula ta może służyć również do porównywania wydajności zapytań

#### Przykład:
```sql
EXPLAIN SELECT
   *
FROM
   film
```

## Funkcje agregujące

Funkcje agregujące wykonują obliczenia na zestawie wierszy i zwracają pojedynczy wiersz. PostgreSQL udostępnia wszystkie standardowe funkcje agregujące SQL w następujący sposób:
- AVG () - zwraca średnią wartość.
- COUNT () - zwraca liczbę wartości.
- MAX () - zwraca maksymalną wartość.
- MIN () - zwraca minimalną wartość.
- SUM () - zwraca sumę wszystkich lub różnych wartości.

Pełna lista funkcji agregującej: https://www.postgresql.org/docs/9.5/functions-aggregate.html

Często używamy funkcji agregujących z klauzulą GROUP BY w instrukcji SELECT. W tych przypadkach klauzula GROUP BY dzieli zestaw wyników na grupy wierszy i funkcja agregująca wykonuje obliczenia dla każdej grupy, np. maksimum, minimum, średnia itp. Funkcji agregujących można używać funkcji agregujących jako wyrażeń tylko w następujących klauzulach: SELECT i HAVING.

### GROUP BY
Klauzula GROUP BY dzieli wiersze zwrócone z instrukcji SELECT na grupy. Dla  każdej grupy można zastosować funkcję agregującą, np. SUM aby obliczyć sumę pozycji lub
COUNT aby uzyskać liczbę elementów w grupach.

Poniższa instrukcja ilustruje składnię klauzuli GROUP BY:
```sql
SELECT 
    column_1, 
    aggregate_function(column_2)
FROM 
    tbl_name
GROUP BY 
    column_1;
```
Klauzula GROUP BY musi pojawić się zaraz po klauzuli FROM lub WHERE, n0astępnie GROUP BY zawiera listę  kolumna oddzielonych przecinkami. 

### HAVING
Często używamy klauzuli HAVING w połączeniu z klauzulą GROUP BY do filtrowania wierszy grup
które nie spełniają określonego warunku.

Poniższa instrukcja ilustruje typową składnię klauzuli HAVING:
```sql
SELECT
    column_1,
    aggregate_function (column_2)
FROM
    tbl_name
GROUP BY
    column_1
HAVING
    condition;
```
Klauzula HAVING ustawia warunek dla wierszy grup utworzonych przez klauzulę GROUP BY.  

Klauzula GROUP BY ma zastosowanie, podczas gdy klauzula WHERE określa wcześniej warunki dla poszczególnych wierszy.

## Zadania wprowadzające
Wykonaj zapytania przy użyciu DBMS:  
  
1. Znajdź listę wszystkich filmów o tej samej długości.
2. Znajdź wszystkich klientów mieszkających w tym samym mieście.
3. Oblicz średni koszt wypożyczenia wszystkich filmów.
4. Oblicz i wyświetl liczbę filmów we wszystkich kategoriach.
5. Wyświetl liczbę wszystkich klientów pogrupowanych według kraju.
6. Wyświetl informacje o sklepie, który ma więcej niż 100 klientów i mniej niż 300 klientów.
7. Wybierz wszystkich klientów, którzy oglądali filmy ponad 200 godzin.
8. Oblicz średnią wartość wypożyczenia filmu.
9. Oblicz średnią wartość długości filmu we wszystkich kategoriach.
10. Znajdź najdłuższe tytuły filmowe we wszystkich kategoriach.
11. Znajdź najdłuższy film we wszystkich kategoriach. Porównaj wynik z pkt 10.

In [1]:
import psycopg2 as pg
import pandas as pd

connection = pg.connect(host='pgsql-196447.vipserv.org', port=5432, dbname='wbauer_adb', user='wbauer_adb', password='adb2020');

#### 1. Znajdź listę wszystkich filmów o tej samej długości.

In [2]:
df1 = pd.read_sql("""
SELECT
    f1.film_id film_1_id, 
    f1.title film_1_title,
    f1.length film_1_length,
    f2.film_id film_2_id,
    f2.title film_2_title,
    f2.length film_2_length
FROM
    film f1
INNER JOIN
    film f2 ON f1.length=f2.length
WHERE
    f1.title != f2.title;
""", con=connection)
df1

,film_1_id,film_1_title,film_1_length,film_2_id,film_2_title,film_2_length
0,730,Ridgemont Submarine,46,469,Iron Moon,46
1,730,Ridgemont Submarine,46,15,Alien Center,46
2,730,Ridgemont Submarine,46,504,Kwai Homeward,46
3,730,Ridgemont Submarine,46,505,Labyrinth League,46
4,469,Iron Moon,46,730,Ridgemont Submarine,46
...,...,...,...,...,...,...
6967,182,Control Anthem,185,690,Pond Seattle,185
6968,182,Control Anthem,185,212,Darn Forrester,185
6969,182,Control Anthem,185,872,Sweet Brotherhood,185
6970,182,Control Anthem,185,817,Soldiers Evolution,185


#### 2. Znajdź wszystkich klientów mieszkających w tym samym mieście.

In [3]:
df2 = pd.read_sql("""
SELECT
    c1.customer_id customer_1_id, 
    c1.first_name || ' ' || c1.last_name customer_1_name,
    c1.address_id customer_address_id,
    c2.customer_id customer_2_id, 
    c2.first_name || ' ' || c2.last_name customer_2_name,
    c2.address_id customer_2_address_id
FROM
    customer c1
INNER JOIN
    customer c2 ON(
        SELECT 
            address.city_id 
        FROM 
            address 
        WHERE 
            address_id = c1.address_id ) = (
        SELECT 
            address.city_id 
        FROM 
            address 
        WHERE 
            address_id = c2.address_id)
WHERE
    c1.customer_id != c2.customer_id
""", con=connection)
df2

,customer_1_id,customer_1_name,customer_address_id,customer_2_id,customer_2_name,customer_2_address_id
0,330,Scott Shelley,335,537,Clinton Buford,543
1,537,Clinton Buford,543,330,Scott Shelley,335
2,512,Cecil Vines,517,252,Mattie Hoffman,256
3,252,Mattie Hoffman,256,512,Cecil Vines,517


#### 3. Oblicz średni koszt wypożyczenia wszystkich filmów.

In [4]:
df3 = pd.read_sql("""
SELECT
    AVG(rental_rate)
FROM
    film;
""", con=connection)
df3

,avg
0,2.98


#### 4. Oblicz i wyświetl liczbę filmów we wszystkich kategoriach.

In [5]:
df4 = pd.read_sql("""
SELECT
    category_id,
    name,
    COUNT(film_category.film_id)
FROM
    category
INNER JOIN
    film_category USING(category_id)
GROUP BY
    category_id
ORDER BY
    category_id ASC
""", con=connection)
df4

,category_id,name,count
0,1,Action,64
1,2,Animation,66
2,3,Children,60
3,4,Classics,57
4,5,Comedy,58
5,6,Documentary,68
6,7,Drama,62
7,8,Family,69
8,9,Foreign,73
9,10,Games,61


#### 5. Wyświetl liczbę wszystkich klientów pogrupowanych według kraju.

In [6]:
df5 = pd.read_sql("""
SELECT
    country_id,
    country,
    COUNT(customer_id)
FROM
    country
INNER JOIN
    city USING(country_id)
INNER JOIN
    address USING(city_id)
INNER JOIN
    customer USING(address_id)
GROUP BY
    country_id
ORDER BY
    country_id ASC
""", con=connection)
df5

,country_id,country,count
0,1,Afghanistan,1
1,2,Algeria,3
2,3,American Samoa,1
3,4,Angola,2
4,5,Anguilla,1
...,...,...,...
103,105,Vietnam,6
104,106,"Virgin Islands, U.S.",1
105,107,Yemen,4
106,108,Yugoslavia,2


#### 6. Wyświetl informacje o sklepie, który ma więcej niż 100 klientów i mniej niż 300 klientów.

In [7]:
df6 = pd.read_sql("""
SELECT
    store_id,
    manager_staff_id,
    store.address_id,
    store.last_update
FROM
    store
INNER JOIN
    customer USING(store_id)
GROUP BY
    store_id
HAVING
    COUNT(customer_id) > 100 AND COUNT(customer_id) < 300
ORDER BY
    store_id ASC
""", con=connection)
df6

,store_id,manager_staff_id,address_id,last_update
0,2,2,2,2006-02-15 09:57:12


#### 7. Wybierz wszystkich klientów, którzy oglądali filmy ponad 200 godzin.

In [8]:
df7 = pd.read_sql("""
SELECT
    DISTINCT customer_id,
    first_name || ' ' || last_name customer_name
FROM
    customer
INNER JOIN
    rental USING(customer_id)
WHERE
    (return_date-rental_date) > INTERVAL '200 hours'
ORDER BY
    customer_id ASC
""", con=connection)
df7

,customer_id,customer_name
0,1,Mary Smith
1,2,Patricia Johnson
2,3,Linda Williams
3,4,Barbara Jones
4,5,Elizabeth Brown
...,...,...
561,595,Terrence Gunderson
562,596,Enrique Forsythe
563,597,Freddie Duggan
564,598,Wade Delvalle


#### 8. Oblicz średnią wartość wypożyczenia filmu.

In [9]:
df8 = pd.read_sql("""
SELECT
    AVG(amount)
FROM
    payment;
""", con=connection)
df8

,avg
0,4.200606


#### 9. Oblicz średnią wartość długości filmu we wszystkich kategoriach.

In [10]:
df9 = pd.read_sql("""
SELECT
    category_id,
    name,
    AVG(film.length)
FROM
    category
INNER JOIN
    film_category USING(category_id)
INNER JOIN
    film USING(film_id)
GROUP BY
    category_id
ORDER BY
    category_id ASC
""", con=connection)
df9

,category_id,name,avg
0,1,Action,111.609375
1,2,Animation,111.015152
2,3,Children,109.800000
3,4,Classics,111.666667
4,5,Comedy,115.827586
5,6,Documentary,108.750000
6,7,Drama,120.838710
7,8,Family,114.782609
8,9,Foreign,121.698630
9,10,Games,127.836066


#### 10. Znajdź najdłuższe tytuły filmowe we wszystkich kategoriach.

In [11]:
df10 = pd.read_sql("""
SELECT 
    data.category_id, 
    data.name, 
    data.max_title_length, 
    film.title 
FROM
    (SELECT 
        category_id, 
        name, 
        MAX(LENGTH(film.title)) max_title_length
    FROM 
        category
    INNER JOIN 
        film_category USING(category_id)
    INNER JOIN 
        film USING(film_id)
    GROUP BY 
        category_id
    ORDER BY 
        category_id ASC
    ) data
INNER JOIN 
    film ON (data.max_title_length = LENGTH(film.title) AND data.category_id = (SELECT 
                                                                                f_c_2.category_id
                                                                            FROM
                                                                                film_category f_c_2
                                                                            WHERE 
                                                                                f_c_2.film_id = film.film_id))
ORDER BY 
    data.category_id ASC
""", con=connection)
df10

,category_id,name,max_title_length,title
0,1,Action,23,Entrapment Satisfaction
1,2,Animation,22,Telemark Heartbreakers
2,3,Children,20,Heartbreakers Bright
3,3,Children,20,Microcosmos Paradise
4,3,Children,20,Sweethearts Suspects
5,4,Classics,23,Extraordinary Conquerer
6,5,Comedy,23,Trainspotting Strangers
7,6,Documentary,22,Intolerable Intentions
8,6,Documentary,22,Deliverance Mulholland
9,7,Drama,22,Goldfinger Sensibility


#### 11. Znajdź najdłuższy film we wszystkich kategoriach. Porównaj wynik z pkt 10.

In [12]:
df11 = pd.read_sql("""
SELECT 
    data.category_id, 
    data.name, 
    data.max_film_length, 
    film.title 
FROM
    (SELECT 
        category_id, 
        name, 
        MAX(film.length) max_film_length
    FROM 
        category
    INNER JOIN 
        film_category USING(category_id)
    INNER JOIN 
        film USING(film_id)
    GROUP BY 
        category_id
    ORDER BY 
        category_id ASC
    ) data
INNER JOIN 
    film ON (data.max_film_length = film.length AND data.category_id = (SELECT 
                                                                                f_c_2.category_id
                                                                            FROM
                                                                                film_category f_c_2
                                                                            WHERE 
                                                                                f_c_2.film_id = film.film_id))
ORDER BY 
    data.category_id ASC
""", con=connection)
df11

,category_id,name,max_film_length,title
0,1,Action,185,Darn Forrester
1,1,Action,185,Worst Banger
2,2,Animation,185,Pond Seattle
3,2,Animation,185,Gangs Pride
4,3,Children,178,Fury Murder
5,3,Children,178,Wrong Behavior
6,4,Classics,184,Conspiracy Spirit
7,5,Comedy,185,Control Anthem
8,6,Documentary,183,Young Language
9,6,Documentary,183,Wife Turn


W pkt. 10 przeszukujemy bazę pod względem znalezienia filmów z tytułami o największej ilości znaków w danej kategorii.
Zwrócone wartości długości tytułów zawierają sie między 20 a 27.

W pkt. 11 przeszukujemy bazę pod względem znalezienia filmów o najdłuższym czasie trwania w danej kategorii.
Zwrócone wartości długości filmów zawierają sie między 178 a 185.

## Zadanie implementacyjne
Zaimplementuj wszystkie funkcje w pliku main.py zgodnie z opisem a następnie przetestuj je w notatniku.

In [13]:
!pytest -q test_main.py

...................................                                      [100%]
============================== warnings summary ===============================
..\..\..\..\..\Anaconda\envs\marcinbereznicki\lib\site-packages\pyreadline\py3k_compat.py:8
  E:\Programowanie\Anaconda\envs\marcinbereznicki\lib\site-packages\pyreadline\py3k_compat.py:8: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated since Python 3.3, and in 3.10 it will stop working
    return isinstance(x, collections.Callable)

-- Docs: https://docs.pytest.org/en/stable/warnings.html
35 passed, 1 warning in 2.52s


In [14]:
import main

result = main.film_in_category(1)
result

,title,languge,category
0,Amadeus Holy,English,Action
1,American Circus,English,Action
2,Antitrust Tomatoes,English,Action
3,Ark Ridgemont,English,Action
4,Barefoot Manchurian,English,Action
...,...,...,...
59,Uprising Uptown,English,Action
60,Waterfront Deliverance,English,Action
61,Werewolf Lola,English,Action
62,Women Dorado,English,Action


In [15]:
result = main.number_films_in_category(1)
result

,category,count
0,Action,64


In [16]:
result = main.number_film_by_length(50, 100)
result

,length,count
0,50,9
1,51,7
2,52,7
3,53,9
4,54,6
5,55,2
6,56,5
7,57,7
8,58,7
9,59,9


In [17]:
result = main.client_from_city('Athenai')
result

,city,first_name,last_name
0,Athenai,Linda,Williams


In [18]:
result = main.avg_amount_by_length(48)
result

,length,avg
0,48,4.295389


In [19]:
result = main.client_by_sum_length(1264)
result

,first_name,last_name,sum
0,Brian,Wyman,1265
1,Antonio,Meek,1451
2,Leona,Obrien,1588
3,Katherine,Rivera,1615
4,Tiffany,Jordan,1667
...,...,...,...
594,Wesley,Bull,4808
595,Clara,Shaw,4808
596,Tammy,Sanders,5065
597,Eleanor,Hunt,5360


In [20]:
result = main.category_statistic_length('Action')
result

,category,avg,sum,min,max
0,Action,111.609375,7143,47,185
